In [11]:
# Import Required Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, EfficientNetB0, VGG16
from tensorflow.keras.models import Model

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, auc, roc_auc_score, RocCurveDisplay
from sklearn.preprocessing import label_binarize

import keras_tuner as kt
from tensorflow.keras.optimizers import Adam
from keras_tuner import HyperParameters

In [12]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [14]:
# Enable Mixed Precision for Faster Training
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

# Define Constants
TRAIN_DIR = "PROCESSED_DATA/TRAINING_DATA/TRAINING_AUGMENTED_DATA"
VALID_DIR = "PROCESSED_DATA/VALIDATION_DATA/"
TEST_DIR = "PROCESSED_DATA/TEST_DATA/"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NORMALIZE_FLAG = True

# Data Generators
NORM_DATAGEN = ImageDataGenerator(rescale=1./255)
NO_FRILLS_DATAGEN = ImageDataGenerator()

# Data Loading Function
def load_data(directory, shuffle_flag=True):
    generator = NORM_DATAGEN.flow_from_directory(
        directory,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=shuffle_flag
    ) if NORMALIZE_FLAG else NO_FRILLS_DATAGEN.flow_from_directory(
        directory,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=shuffle_flag
    )
    return generator

# Load Data
TRAIN_GENERATOR = load_data(TRAIN_DIR)
VAL_GENERATOR = load_data(VALID_DIR)
TEST_GENERATOR = load_data(TEST_DIR, shuffle_flag=False)

hp = HyperParameters()

# Model Building Function
def build_tunable_cnn(hp):
    model = Sequential([
        Input(shape=(224, 224, 3)),
        Conv2D(32, (3,3), activation='relu', padding='same'),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu', padding='same'),
        MaxPooling2D(2,2),
        Conv2D(128, (3,3), activation='relu', padding='same'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(hp.Float("dropout", min_value=0.2, max_value=0.5, step=0.1)), 
        Dense(TRAIN_GENERATOR.num_classes, activation='softmax', dtype='float32')
    ])
    
    learning_rate = hp.Choice('lr', values=[1e-2, 1e-3, 1e-4])
    batch_size = hp.Choice('batch_size', values=[16, 32])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Hyperparameter Tuning Function
def build_best_model():
    tuner = kt.BayesianOptimization(
        build_tunable_cnn,
        objective='val_accuracy',
        max_trials=20,
        executions_per_trial=1,
        directory='bayesian_tuning',
        project_name='lr_and_drop_tuning'
    )
    
    tuner.search(TRAIN_GENERATOR, validation_data=VAL_GENERATOR, epochs=10)
    
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_hps_dict = {
        'best_lr': best_hps.get('lr'),
        'best_dropout': best_hps.get('dropout'),
        'best_batch_size': best_hps.get('batch_size')
    }
    
    best_model = tuner.hypermodel.build(best_hps)
    best_model_training_history = best_model.fit(
        TRAIN_GENERATOR, 
        validation_data=VAL_GENERATOR, 
        epochs=10, 
        batch_size=best_hps.get('batch_size')
    )

    return best_hps_dict, best_model, best_model_training_history

# Model Evaluation Function
def evaluate_model(model, filename="best_model.h5"):
    test_loss, test_acc = model.evaluate(TEST_GENERATOR)
    print(f"Test Accuracy: {test_acc:.4f}")
    
    model.save(filename)
    print(f"Model saved as {filename}")

    return test_loss, test_acc


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3080 Ti Laptop GPU, compute capability 8.6
Found 6642 images belonging to 11 classes.
Found 440 images belonging to 11 classes.
Found 220 images belonging to 11 classes.


In [15]:
# Main Training Function
def main():

    with tf.device('/GPU:0'):  # Forces execution on GPU 0
        best_hps_dict, best_model, best_model_training_history = build_best_model()
        print(f'Best Hyperparameters:\n {best_hps_dict}')
        
        test_loss, test_acc = evaluate_model(best_model)
        print(f'Test Loss: {test_loss}, Test Accuracy: {test_acc}')
        
        best_model.save("simple_cnn_best_model_bayes_optimization.h5")
        print("Training complete!")

# Run Training
if __name__ == "__main__":
    main()

Trial 20 Complete [00h 01m 32s]
val_accuracy: 0.46590909361839294

Best val_accuracy So Far: 0.4749999940395355
Total elapsed time: 00h 32m 11s
Epoch 1/10
208/208 [==============================] - 10s 42ms/step - loss: 2.2924 - accuracy: 0.1721 - val_loss: 2.1886 - val_accuracy: 0.2409
Epoch 2/10
208/208 [==============================] - 9s 41ms/step - loss: 2.0362 - accuracy: 0.3022 - val_loss: 2.0563 - val_accuracy: 0.3159
Epoch 3/10
208/208 [==============================] - 8s 40ms/step - loss: 1.7908 - accuracy: 0.3978 - val_loss: 1.9219 - val_accuracy: 0.3795
Epoch 4/10
208/208 [==============================] - 8s 39ms/step - loss: 1.5927 - accuracy: 0.4714 - val_loss: 1.8795 - val_accuracy: 0.3818
Epoch 5/10
208/208 [==============================] - 8s 39ms/step - loss: 1.3965 - accuracy: 0.5388 - val_loss: 1.8880 - val_accuracy: 0.4318
Epoch 6/10
208/208 [==============================] - 8s 40ms/step - loss: 1.2094 - accuracy: 0.6072 - val_loss: 1.9581 - val_accuracy: 0.44